In [358]:
import numpy as np
import math
import os, glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from matplotlib.ticker import MultipleLocator

In [364]:
mapset_path = '../dataset/Analiza zbioru danych/mapset'
a_star_path = 'dataset/Auto_mapset/A_star'

In [ ]:


def slope_analyze(ds):
    iter_range = len(ds.index)
    for i in range(iter_range):
        slope_path = ds.iloc[i]['Slope aspect path']
        with open(slope_path, 'rb') as f:
            slope = np.load(f)
        slope_arr = dem.flatten()
     #   dem_diff = np.max(dem_arr)-np.mean(dem_arr)
        if i == 0:
            slope_hist = slope_arr
        else:
            slope_hist=np.append(slope_hist, slope_arr)
    return slope_hist

def dem_analyze(ds):
    iter_range = len(ds.index)
    for i in range(iter_range):
        dem_path = ds.iloc[i]['DEM path']
        with open(dem_path, 'rb') as f:
            dem = np.load(f)
        dem_arr = dem.flatten()
        dem_diff = np.max(dem_arr)-np.min(dem_arr)
        if i == 0:
            dem_hist = dem_diff
        else:
            dem_hist=np.append(dem_hist, dem_diff)
    return dem_hist
   
def calc_dist(ds, res=1.5):
    s_x = ds['Start x coord'].to_numpy(dtype=np.float32, copy=True)
    s_y = ds['Start y coord'].to_numpy(dtype=np.float32, copy=True)
    g_x = ds['Goal x coord'].to_numpy(dtype=np.float32, copy=True)
    g_y = ds['Goal y coord'].to_numpy(dtype=np.float32, copy=True)
    
    s = np.stack((s_x,s_y), axis=1)
    g = np.stack((g_x,g_y), axis=1)
    
    dist = np.sqrt(np.power((g[:,0]-s[:,0]),2)+np.power((g[:,1]-s[:,1]),2))*res
    return dist

def to_percent(y, position):
    # Ignore the passed in position. This has the effect of scaling the default
    # tick locations.
    global n

    s = str(round(100 * y / n, 3))
    print (y)

    # The percent symbol needs escaping in latex
    if matplotlib.rcParams['text.usetex'] is True:
        return s + r'$\%$'
    else:
        return s + '%'
    
def plot_histograms(ds, save_fig=True, export_path='../dataset/Analiza zbioru danych/figures', name='mapset_500x500'):

    dem_hist = dem_analyze(ds)
    slope_hist = slope_analyze(ds)
    dist = calc_dist(ds)

    with plt.style.context('ggplot'):
        fig, ax = plt.subplots(figsize=(16,12), dpi=144)
        plt.clf()
    #    fig.suptitle('Analiza własności terenowych map w zbiorze', fontsize=14)
        ax_1 = fig.add_subplot(221)
        n, bins, patches = ax_1.hist(dem_hist, bins=20, facecolor = '#2ab0ff', edgecolor='white', linewidth=0.5)
        plt.xlabel('Zróżnicowanie wysokości [m]', fontsize=9)
        plt.ylabel('Ilość przypadków testowych [-]', fontsize=9)
        plt.grid(True,'minor', 'x',  linewidth=0.2, color='white',linestyle='-')
        plt.grid(True,'major')
        plt.title('Zróżnicowanie wysokości terenu', fontsize=10)
        ax_1.minorticks_on()

        ax_2 = fig.add_subplot(212)
        n, bins, patches = ax_2.hist(slope_hist, density=True, bins=20, facecolor = '#E42AFF', edgecolor='white', linewidth=0.5)
        plt.xlabel('Nachylenie terenu [deg]', fontsize=9)
        plt.grid(True,'minor', 'x',  linewidth=0.2, color='white',linestyle='-')
        plt.grid(True,'major')
        plt.ylabel('Ilość pikseli [%]', fontsize=9)
        ax_2.minorticks_on()
        plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
        plt.title('Charakter rzeźby terenu', fontsize=10)
        
        ax_3 = fig.add_subplot(222)
        n, bins, patches = ax_3.hist(dist, bins=10, facecolor = '#FF792A', edgecolor='white', linewidth=0.5)
        plt.grid(True,'minor', 'x',  linewidth=0.2, color='white',linestyle='-')
        plt.grid(True,'major')
        plt.ylabel('Ilość przypadków testowych [-]', fontsize=9)
        plt.xlabel('Odległość punktów na trasie [m]', fontsize=9)
        ax_3.minorticks_on()
        ax_3.grid(True,which='minor', axis='x',  linewidth=0.3, color='white',linestyle='-')
#         ax_3.yaxis.set_major_locator(MultipleLocator(1))
        ax_3.xaxis.set_major_locator(MultipleLocator(300))
        plt.title('Odległość zadanych punktów startu/mety', fontsize=10)
        
        if save_fig:
            name = name + '.jpg'
            fig_path = os.path.join(export_path, name)
            plt.savefig(fig_path)
            plt.close()

In [384]:
def get_mapsets(mapset_path):
    search_exp = os.path.join(mapset_path, '*.csv')
    files = []
    for file in glob.glob(search_exp):
        files.append(file)
    return files

def iterate_over_mapset(mapset_path):
    files = get_mapsets(mapset_path)
    
    for file in files:
        file_name = os.path.split(file)[1]
        file_name = os.path.splitext(file_name)[0]
        ds = pd.read_csv(file)
        plot_histograms(ds, save_fig=True, export_path='../dataset/Analiza zbioru danych/figures', name=file_name)

In [387]:
iterate_over_mapset(mapset_path)

mapset_1000x1000 ../dataset/Analiza zbioru danych/mapset\mapset_1000x1000.csv
mapset_2000x2000 ../dataset/Analiza zbioru danych/mapset\mapset_2000x2000.csv
mapset_250x250 ../dataset/Analiza zbioru danych/mapset\mapset_250x250.csv
mapset_500x500 ../dataset/Analiza zbioru danych/mapset\mapset_500x500.csv
